## TO DO

2. filter vcf lines to return important bits

3. check reference chromosome

4. pull data from gene and variant databases (prefereably already hosted online)

In [21]:
using JuliaDB
using IndexedTables
using Dagger
using VCFTool

# For viewing DataFrames

ENV["COLUMNS"]=100

ENV["LINES"]=200

# For viewing JuliaDB tables and arrays

IndexedTables.set_show_compact!(false);

In [31]:
input_dir = "../input/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_738_1000_file_path = joinpath(input_dir, "738_variants_1000.vcf.gz")

vcf_769_3_file_path = joinpath(input_dir, "769_3_variants.vcf.gz")

vcf_test_file_path = joinpath(input_dir, "test.vcf.gz")

vcf_file_path_to_use = vcf_738_1000_file_path;

In [32]:
vcf_table = make_vcf_indexedtable(vcf_file_path_to_use)

Table with 999 rows, 10 columns:
CHROM  POS     ID   REF            ALT      QUAL    FILTER                                INFO                                                FORMAT                                     GERM
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
1      10439   "."  "AC"           "A"      "72"    "PASS"                                "CIGAR=1M1D;RU=C;REFREP=4;IDREP=3;MQ=9"             "GT:GQ:GQX:DPI:AD:ADF:ADR:FT:PL"           "0/1:31:5:7:2,5:0,1:2,4:PASS:108,0,28"
1      13284   "."  "G"            "A"      "60"    "LowGQX;NoPassedVariantGTs"           "SNVHPOL=4;MQ=17"                                   "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL"     "0/1:93:1:30:1:19,11:12,8:7,3:-9.2:LowGQX:95,0,157"
1      13868   "."  "A"            "G"      "1"     "LowGQX;LowDepth

In [33]:
# https://www.snpedia.com/index.php/Rs1042522
rs1042522 = ["chr17", 7676154]

# TP53
tp53 = ["chr17", 7661779:7687550]

# test set
t = ["chr1", 10000:12000]

2-element Array{Any,1}:
 "chr1"
 10000:12000

## Search for variant

In [42]:
filter_variant = filter(i -> (i.CHROM == 1) && (i.POS == 13868), vcf_table)

Table with 1 rows, 10 columns:
CHROM  POS    ID   REF  ALT  QUAL  FILTER                                INFO              FORMAT                                  GERM
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
1      13868  "."  "A"  "G"  "1"   "LowGQX;LowDepth;NoPassedVariantGTs"  "SNVHPOL=3;MQ=4"  "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL"  "0/1:3:0:1:0:0,1:0,0:0,1:0.0:LowGQX;LowDepth:28,3,0"

## Search for region

In [50]:
filter_region = filter(i -> (i.CHROM == 1) && (i.POS > 10000) && (i.POS < 20000), vcf_table)

Table with 11 rows, 10 columns:
CHROM  POS    ID   REF   ALT   QUAL   FILTER                                INFO                                      FORMAT                                  GERM
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
1      10439  "."  "AC"  "A"   "72"   "PASS"                                "CIGAR=1M1D;RU=C;REFREP=4;IDREP=3;MQ=9"   "GT:GQ:GQX:DPI:AD:ADF:ADR:FT:PL"        "0/1:31:5:7:2,5:0,1:2,4:PASS:108,0,28"
1      13284  "."  "G"   "A"   "60"   "LowGQX;NoPassedVariantGTs"           "SNVHPOL=4;MQ=17"                         "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL"  "0/1:93:1:30:1:19,11:12,8:7,3:-9.2:LowGQX:95,0,157"
1      13868  "."  "A"   "G"   "1"    "LowGQX;LowDepth;NoPassedVariantGTs"  "SNVHPOL=3;MQ=4"                          "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL"  "0/1:3:0:1:0:0,1:0,0:0,1:0.

In [56]:
typeof(filter_region)

IndexedTable{StructArrays.StructArray{NamedTuple{(:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),Tuple{Any,Any,String,String,String,String,String,String,String,String}},1,NamedTuple{(:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),Tuple{Array{Any,1},Array{Any,1},WeakRefStrings.StringArray{String,1},WeakRefStrings.StringArray{String,1},WeakRefStrings.StringArray{String,1},WeakRefStrings.StringArray{String,1},WeakRefStrings.StringArray{String,1},WeakRefStrings.StringArray{String,1},WeakRefStrings.StringArray{String,1},WeakRefStrings.StringArray{String,1}}},Int64}}

## Get Genotype

In [61]:
function get_genotype_of_vcf_indexedtable(vcf_table)

    genotypes = []

    for row in vcf_table

        genotype = []

        allele_ref = row[:REF]

        allele_alt = row[:ALT]

        println(allele_alt)

        println(row[:GERM])

        numeric_genotype = split(split(row[:GERM], ":")[1], "/")

        println(numeric_genotype)

        if occursin("|", numeric_genotype[1])

            numeric_genotype = split(numeric_genotype[1], "|")

        end

        counter = 1

            while counter < 3

                if "0" == numeric_genotype[counter]

                    push!(genotype, allele_ref)

                elseif "1" == numeric_genotype[counter]

                    push!(genotype, split(allele_alt, ",")[1])

                elseif "2" == numeric_genotype[counter]

                    push!(genotype, split(allele_alt, ",")[2])

                end    

            counter += 1

            end

        push!(genotypes, genotype)

    end

return genotypes

A
0/1:31:5:7:2,5:0,1:2,4:PASS:108,0,28
SubString{String}["0", "1"]
A
0/1:93:1:30:1:19,11:12,8:7,3:-9.2:LowGQX:95,0,157
SubString{String}["0", "1"]
G
0/1:3:0:1:0:0,1:0,0:0,1:0.0:LowGQX;LowDepth:28,3,0
SubString{String}["0", "1"]
G
0/1:3:0:1:0:0,1:0,0:0,1:0.0:LowGQX;LowDepth:27,3,0
SubString{String}["0", "1"]
GC
1/1:24:17:9:0,9:0,9:0,0:PASS:255,27,0
SubString{String}["1", "1"]
G
0/1:20:0:3:1:2,1:0,0:2,1:0.0:LowGQX:23,0,42
SubString{String}["0", "1"]
G
0/1:16:0:2:2:1,1:0,0:1,1:0.0:LowGQX;LowDepth:21,0,20
SubString{String}["0", "1"]
G
0/1:22:0:6:10:4,2:0,0:4,2:0.0:LowGQX:25,0,58
SubString{String}["0", "1"]
G
0/1:33:0:6:9:3,3:0,0:3,3:0.0:LowGQX:37,0,35
SubString{String}["0", "1"]
T
0/1:15:0:17:0:14,3:10,1:4,2:0.0:LowGQX:17,0,146
SubString{String}["0", "1"]
C
0/1:47:1:11:0:9,2:2,1:7,1:-3.2:LowGQX:48,0,130
SubString{String}["0", "1"]


11-element Array{Any,1}:
 Any["AC", "A"]
 Any["G", "A"]
 Any["A", "G"]
 Any["A", "G"]
 Any["GC", "GC"]
 Any["T", "G"]
 Any["T", "G"]
 Any["C", "G"]
 Any["T", "G"]
 Any["C", "T"]
 Any["T", "C"]

## Tabix Regions

In [ ]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)